In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import seaborn as sns
import rasterio as rio
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
import disco_tif.process_sb_tiff
import disco_tif.hillshades_pca
#import disco_tif.geotiff_plotting
import disco_tif.write_mapfile

In [4]:
import datetime
def nowTime():
    return datetime.datetime.now().strftime("%H:%M:%S")
def now():
    return datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
def snow():
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
def today():
    return datetime.datetime.now().strftime("%Y-%m-%d")
def stoday():
    return datetime.datetime.now().strftime("%Y%m%d")

In [5]:
tiff_dir = os.getcwd()

single_band_tiff_path = os.path.abspath('../example_data/E020N40/E020N40.tif')

single_band_tiff_path

'/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40.tif'

In [6]:
method_list = ['pseudo_linear', 'pseudo_hist_norm']
for method in method_list:
    ph_colormap, ph_data_breaks, ph_new_file_paths = disco_tif.process_sb_tiff.make_rgba_tiff_from_single_Band(single_band_tiff_path,
                                                                                                               #data_min_max=datarange,
                                                                                                               cmap_method=method,
                                                                                                               )



GIS software compatible color look up table successfully written to: 
	- /Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_EMeraldCustomTerrain_-28_to_2540_pseudo_linear_qgis_color_table_8-colors_20240517.txt

New single-channel colorized geotiff written to: 
	- '/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_EMeraldCustomTerrain_-28_to_2540_pseudo_linear.tif'


GIS software compatible color look up table successfully written to: 
	- /Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_EMeraldCustomTerrain_-28_to_2540_pseudo_hist_norm_qgis_color_table_8-colors_20240517.txt

New single-channel colorized geotiff written to: 
	- '/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_EMeraldCustomTerrain_-28_to_2540_pseudo_hist_norm.tif'



In [7]:
hs_azimuths, hs_altitudes = disco_tif.hillshades_pca.build_hs_az_al(start_az=0, num_az_angles=8)

azimuths = [0, 45, 90, 135, 180, 225, 270, 315]
altitudes = [30, 60]


In [8]:
hs_azimuths, hs_altitudes = disco_tif.hillshades_pca.build_hs_az_al(start_az=0, num_az_angles=4, num_al_angles=1)

azimuths = [0, 90, 180, 270]
altitudes = [30]


In [9]:
rasters_dict, filepaths_dict = disco_tif.hillshades_pca.build_hillshade(single_band_tiff_path=single_band_tiff_path,
                                                                        data_min_max=[min(ph_data_breaks), max(ph_data_breaks)],
                                                                        hs_azimuths=hs_azimuths,
                                                                        hs_altitudes=hs_altitudes,
                                                                        cmap=ph_colormap,
                                                                        process_pca=True)

New single-channel geotiff generated successfully: '/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_hillshade_az000_al30.tif'
New single-channel geotiff generated successfully: '/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_hillshade_az090_al30.tif'
New single-channel geotiff generated successfully: '/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_hillshade_az180_al30.tif'
New single-channel geotiff generated successfully: '/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_hillshade_az270_al30.tif'
New single-channel geotiff generated successfully: '/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_hillshade_pca4-component_1.tif'
New single-channel geotiff generated successfully: '/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_hillshade_pca4-component_2.tif'
New single-channel geotiff generated successfully: '/Users/bbloss/EMeraldCodes/disco-tif/example_data/E020N40/E020N40_hillsh

# testing area

In [10]:
for key, value in rasters_dict.items():
    print(f"{key}")
    print(f"\t[{np.nanmin(value)}, {np.nanmax(value)}]")

az000_al30
	[17.149256406630954, 254.77094204141144]
az090_al30
	[17.156267048385654, 254.77094204141144]
az180_al30
	[17.170083124517966, 254.77094204141144]
az270_al30
	[17.147609254184715, 254.77094204141144]
component_1
	[-153.2234396432222, 159.08431021749502]
component_2
	[-153.54098566374233, 158.7682569039348]
component_3
	[-30.57417057082339, 97.36274169121482]
